In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [2]:
model, learning_rate = "AdaptableLSTM", .025
# model, learning_rate = "BasicNN", .0024
# model, learning_rate = "LogisticRegressor", .003
epochs = 1000
seed = 1
include_state = True
estate = True
fullq = True
respond_perc = .50
fullseq = False
insertpreds = True
noise = .1
smooth = 0

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    numValFolds = 5,
    epochsToUpdateLabelMods = 5,
    data_kw={"minw": 2,
            "maxw": 29,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq,
             "full_sequence": fullseq,
             "insert_predictions": insertpreds,
             "one_hot_response_features": True
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 200,
        "lr_step_mult": .9, 
        "lr_step_epochs": 100,
        "opt_kw": {
            "lr": learning_rate,
        },
        "labelSmoothPerc": smooth,
        "gaussianNoiseStd": noise
        
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })


print(len(e.bd.test))
print(len(e.bd.train))

report = e.run()






54
214
0	 train loss: 0.2318 train acc: 51.598% test acc: 51.149%
5	 train loss: 0.2318 train acc: 51.598% test acc: 51.149%
10	 train loss: 0.2205 train acc: 51.598% test acc: 51.149%
15	 train loss: 0.2177 train acc: 51.598% test acc: 51.149%
20	 train loss: 0.2163 train acc: 51.598% test acc: 51.149%
25	 train loss: 0.2159 train acc: 51.598% test acc: 51.149%
30	 train loss: 0.2137 train acc: 51.598% test acc: 51.149%
35	 train loss: 0.2126 train acc: 51.598% test acc: 51.149%
40	 train loss: 0.2103 train acc: 51.598% test acc: 51.149%
45	 train loss: 0.2087 train acc: 52.369% test acc: 51.915%
50	 train loss: 0.2044 train acc: 53.525% test acc: 52.799%
55	 train loss: 0.2007 train acc: 54.248% test acc: 53.329%
60	 train loss: 0.1981 train acc: 54.778% test acc: 53.742%
65	 train loss: 0.1974 train acc: 55.773% test acc: 55.038%
70	 train loss: 0.1947 train acc: 56.207% test acc: 55.451%
75	 train loss: 0.1943 train acc: 56.528% test acc: 55.745%
80	 train loss: 0.1901 train acc: 5

In [3]:

print (np.mean(report['train_metrics'], axis=0))
labels = report["metric_labels"]
print(report['train_metrics'][-1, labels.index("Acc")])
print(report['test_metrics'][-1, labels.index("Acc")])

splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
plt.title("Train/Test Performance Over Training")
plt.legend()
plt.ylabel("Metric")
plt.xlabel("Training Epoch")
plt.savefig("simpleNotebookAccPlot.png")

plt.clf()

[1.36698756e-01 8.90791992e-01 8.79866736e-01 8.38381156e-01
 7.12001261e-01 7.12001261e-01 7.12001261e-01 4.65311713e-01
 6.04538031e-01 8.80970195e-01 6.07252966e-01 6.38619485e-01
 7.50077485e-01 4.65311713e-01 6.04538031e-01 8.80970195e-01
 1.57300000e+03 1.44100000e+03 3.21300000e+03]
0.8101814389228821
0.5262227654457092


<Figure size 640x480 with 0 Axes>